In [325]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [326]:
import re
import pandas as pd
from collections import defaultdict

def get_transaction_count(line):
    pattern = r"transactions = \{([^}]+)\}"
    match = re.search(pattern, line)
    if match:
        return set(re.findall(r"Transaction ([a-fA-F0-9]+)", match.group(1)))
    return set()

def get_timestamp(line):
    pattern = r"^\d+\.\d+"
    match = re.match(pattern, line)
    return float(match.group(0)) if match else None

def get_node_name(line):
    pattern = r"Node ([A-Za-z0-9]+)"
    match = re.search(pattern, line)
    return match.group(1) if match else None

def count_unique_mempool_transactions(file_path, node_number):
    unique_transactions = set()
    with open(file_path, 'r') as file:
        for line in file:
            if f"Node {node_number}" in line and "from mempool" in line:
                unique_transactions.update(re.findall(r"Transaction ([a-fA-F0-9]+)", line))
    return len(unique_transactions)

def extract_slot(message_line):
    """
    Extracts the slot number from an externalize message log line.
    Looks for the pattern "slot <number>".
    """
    pattern = r"slot (\d+)"
    match = re.search(pattern, message_line)
    if match:
        return int(match.group(1))
    return None

def process_log_lines(file_path):
    """
    Extracts **all** SCPExternalize messages per node and stores their relevant details.
    A new column 'Slot' is added by parsing the slot number from the log message.
    """
    data = []

    with open(file_path, 'r') as file:
        for line in file:
            # Only consider lines that contain relevant externalization messages
            if ("appended SCPExternalize message for slot" not in line and
                "adopting externalized value for slot" not in line):
                continue
            
            node_name = get_node_name(line)
            timestamp = get_timestamp(line)
            transactions = get_transaction_count(line)
            slot = extract_slot(line)  # Extract slot directly from the log line

            if node_name:
                data.append({
                    "node name": node_name,
                    "Timestamp of finalisation": timestamp,
                    "Finalised transactions": transactions,
                    "Externalize message": line.strip(),
                    "Slot": slot
                })
    
    # Convert the collected data to a DataFrame
    df = pd.DataFrame(data)
    
    # If any row didn't have a slot parsed, you might want to drop or handle it:
    df = df.dropna(subset=["Slot"])

    # Count the number of finalized transactions for each externalize message
    df["No. of finalised transactions"] = df["Finalised transactions"].apply(len)
    
    # Compute total transactions for each node from mempool logs
    df["total_transactions"] = df["node name"].apply(lambda node: count_unique_mempool_transactions(file_path, node))
    
    # Calculate number of transactions not finalized for each node
    df["no. of transactions not finalised"] = df["total_transactions"] - df["No. of finalised transactions"]

    return df

# Example usage:
# file_path = 'path/to/your/simulator_events_log.txt'
# df = process_log_lines(file_path)
# print(df)


In [327]:
file_path = 'src/simulator_events_log.txt'

df = process_log_lines(file_path)


# df_sorted = df.sort_values(by='Timestamp of finalisation', ascending=True)

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Prevent line wrapping
pd.set_option('display.colheader_justify', 'center')  # Center column headers
pd.set_option('display.float_format', '{:.4f}'.format)  # Format float values

df = df.sort_values(by='Slot', ascending=True)

display(df) 

,node name,Timestamp of finalisation,Finalised transactions,Externalize message,Slot,No. of finalised transactions,total_transactions,no. of transactions not finalised
0,7,10.0300,"{2f6c5d21, de6b5ac9, b41febcb}",10.03 - NODE - INFO - Node 7 appended SCPExter...,1,3,162,159
32,58,10.3000,"{2f6c5d21, de6b5ac9, b41febcb}",10.30 - Node 58 adopting externalized value f...,1,3,156,153
33,23,10.3100,"{2f6c5d21, de6b5ac9, b41febcb}",10.31 - Node 23 adopting externalized value f...,1,3,153,150
34,47,10.3100,"{2f6c5d21, de6b5ac9, b41febcb}",10.31 - Node 47 adopting externalized value f...,1,3,157,154
35,55,10.3100,"{2f6c5d21, de6b5ac9, b41febcb}",10.31 - Node 55 adopting externalized value f...,1,3,169,166
36,6,10.3100,"{2f6c5d21, de6b5ac9, b41febcb}",10.31 - Node 6 adopting externalized value fo...,1,3,154,151
37,36,10.3100,"{2f6c5d21, de6b5ac9, b41febcb}",10.31 - Node 36 adopting externalized value f...,1,3,161,158
38,9,10.3200,"{2f6c5d21, de6b5ac9, b41febcb}",10.32 - Node 9 adopting externalized value fo...,1,3,159,156
39,42,10.3300,"{2f6c5d21, de6b5ac9, b41febcb}",10.33 - Node 42 adopting externalized value f...,1,3,169,166
40,50,10.3500,"{2f6c5d21, de6b5ac9, b41febcb}",10.35 - Node 50 adopting externalized value f...,1,3,161,158


## Analyze Transaction Matches across slots

In [328]:
def analyze_transaction_matches(df):
    tx_occurrences = {}

    for idx, row in df.iterrows():
        node = row['node name']
        slot = row['Slot']
        # row['Finalised transactions'] is a set; iterate through each transaction hash
        for tx in row['Finalised transactions']:
            if tx not in tx_occurrences:
                tx_occurrences[tx] = set()
            tx_occurrences[tx].add((node, slot))
    
    duplicates = {tx: occ for tx, occ in tx_occurrences.items() if len(occ) > 1} 

    return duplicates


duplicates = analyze_transaction_matches(df)
print("Transactions that appear in more than one slot:")
for tx, occ in duplicates.items():
    print(f"Transaction {tx} appears in: {sorted(list(occ))}")


Transactions that appear in more than one slot:
Transaction 2f6c5d21 appears in: [('0', 1), ('1', 1), ('10', 1), ('11', 1), ('12', 1), ('13', 1), ('14', 1), ('15', 1), ('16', 1), ('17', 1), ('18', 1), ('19', 1), ('2', 1), ('20', 1), ('21', 1), ('22', 1), ('23', 1), ('24', 1), ('25', 1), ('26', 1), ('27', 1), ('28', 1), ('29', 1), ('3', 1), ('30', 1), ('31', 1), ('32', 1), ('33', 1), ('34', 1), ('35', 1), ('36', 1), ('37', 1), ('38', 1), ('39', 1), ('4', 1), ('40', 1), ('42', 1), ('43', 1), ('44', 1), ('45', 1), ('46', 1), ('47', 1), ('48', 1), ('49', 1), ('5', 1), ('50', 1), ('51', 1), ('52', 1), ('53', 1), ('54', 1), ('55', 1), ('56', 1), ('57', 1), ('58', 1), ('59', 1), ('6', 1), ('7', 1), ('8', 1), ('9', 1)]
Transaction de6b5ac9 appears in: [('0', 1), ('1', 1), ('10', 1), ('11', 1), ('12', 1), ('13', 1), ('14', 1), ('15', 1), ('16', 1), ('17', 1), ('18', 1), ('19', 1), ('2', 1), ('20', 1), ('21', 1), ('22', 1), ('23', 1), ('24', 1), ('25', 1), ('26', 1), ('27', 1), ('28', 1), ('29',

#### SHOW ALL TRANSACTIONS THAT APPEAR IN MULTIPLE SLOTS FOR THE SAME NODE

In [329]:
def analyze_node_slots_and_multislot_occurrences(df):
    """
    Analyzes the DataFrame to extract:
      - For each node, a dictionary mapping each Slot to the set of finalized transaction hashes 
        that were reported in that slot.
      - For each node, a dictionary mapping transaction hashes (that occur in multiple slots) to the
        set of slot numbers where they appear.
    
    Args:
        df (pandas.DataFrame): A DataFrame with at least the columns:
            - 'node name'
            - 'Slot'
            - 'Finalised transactions' (a set of transaction hashes)
    
    Returns:
        A tuple (node_slots, multi_occurrences) where:
          - node_slots is a dict: { node_name: { slot: set(transaction hashes), ... }, ... }
          - multi_occurrences is a dict: { node_name: { transaction_hash: set(slots) where it appears, ... }, ... }
    """
    # Build a dictionary that records for each node which transactions are finalized in each slot.
    node_slots = {}
    for idx, row in df.iterrows():
        node = row['node name']
        slot = row['Slot']
        txs = row['Finalised transactions']  # expected to be a set of transaction hashes
        if node not in node_slots:
            node_slots[node] = {}
        if slot not in node_slots[node]:
            node_slots[node][slot] = set()
        node_slots[node][slot].update(txs)
    
    # Now, for each node, gather transaction occurrences across slots.
    multi_occurrences = {}
    for node, slot_dict in node_slots.items():
        tx_occurrences = {}  # maps tx -> set of slots where it occurs
        for slot, txs in slot_dict.items():
            for tx in txs:
                if tx not in tx_occurrences:
                    tx_occurrences[tx] = set()
                tx_occurrences[tx].add(slot)
        # Filter to only transactions that appear in more than one slot.
        multi = {tx: slots for tx, slots in tx_occurrences.items() if len(slots) > 1}
        if multi:
            multi_occurrences[node] = multi

    return node_slots, multi_occurrences

# Example usage:
node_slots, multi_occurrences = analyze_node_slots_and_multislot_occurrences(df)

print("Per-node transaction sets by slot:")
for node, slot_info in node_slots.items():
    print(f"Node {node}:")
    for slot, txs in sorted(slot_info.items()):
        print(f"  Slot {slot}: {sorted(list(txs))}")

print("\nTransactions that appear for the same node in multiple slots:")
for node, tx_info in multi_occurrences.items():
    print(f"Node {node}:")
    for tx, slots in tx_info.items():
        print(f"  Transaction {tx} appears in slots: {sorted(list(slots))}")


Per-node transaction sets by slot:
Node 7:
  Slot 1: ['2f6c5d21', 'b41febcb', 'de6b5ac9']
  Slot 2: ['254d45e2', '28ce6b5b', '427fc773', '5f515056', '700e45d5', '84344b20', 'cac4175c', 'fc945ddb']
  Slot 3: ['17d978c3', '2e11957a', '41af6725', '4d02ce26', '53a698f6', '5f55c38b', '9b44ba54', 'a15f5363', 'aba3652f', 'bf822cdd', 'c500b4cc', 'd4c30704']
  Slot 4: ['36593af6', '3bd9b00d', '5af69cb5', '6c45bd38', '77159e95', '872de5ed', '8de8511', '9e6ba50', 'd829c5fd', 'e13aeed5', 'e38b689a']
  Slot 5: ['1dfe7431', '2158fe30', '389d12d', '39ece391', '802571e8', '990b78a1', 'a450cf58', 'ab538315', 'aed77597', 'c064fe1e', 'd8da1063', 'fd67f139']
  Slot 6: ['12d47ee4', '1c7203ff', '1cc7e0f6', '1de63129', '1fe437b7', '25ee72b5', '4322aac2', '4df40625', '4edd03d', '52e00857', '583da881', '5c9ef3d9', '5fb0f0f9', '6bb9f4ee', '7e03f63f', '827f0fee', '82c870de', '84bb7263', '8f538ad0', '9e87f922', 'b3fb0e9c', 'cbb7267f', 'cdff4d5', 'd871ca12', 'de8240e', 'f5bb05dd', 'fb58f6eb']
  Slot 7: ['27e1013a'

In [330]:
import pandas as pd

def analyze_transaction_matches_with_msgs(df):
    """
    Find transactions that show up in more than one slot *and* record
    the first 3 words of the log messages that caused each appearance.
    
    Returns a dict:
      tx_hash -> {
         'occurrences': set of (node, slot),
         'msg_types':   set of first‑3‑word summaries
      }
    """
    tx_occurrences = {}   # tx -> set((node,slot))
    tx_msgtypes   = {}    # tx -> set(msg_type)
    
    for _, row in df.iterrows():
        node    = row['node name']
        slot    = row['Slot']
        txs     = row['Finalised transactions']
        msg     = row['Externalize message']
        # grab the first 3 words of the log line
        msg_type = " ".join(msg.split()[:10])
        
        for tx in txs:
            tx_occurrences.setdefault(tx, set()).add((node, slot))
            tx_msgtypes.setdefault(tx,   set()).add(msg_type)
    
    # now filter to only those that appear in >1 slot
    duplicates = {}
    for tx, occ in tx_occurrences.items():
        if len(occ) > 1:
            duplicates[tx] = {
                'occurrences': occ,
                'msg_types':   tx_msgtypes.get(tx, set())
            }
    return duplicates

# --- example usage ---
duplicates = analyze_transaction_matches_with_msgs(df)

print("Transactions that appear in more than one slot, with their first‐3‐word log types:")
for tx, info in duplicates.items():
    occ       = sorted(info['occurrences'])
    msg_types = sorted(info['msg_types'])
    print(f"- {tx}:")
    print(f"    slots:    {occ}")
    print(f"    messages: {msg_types}")


Transactions that appear in more than one slot, with their first‐3‐word log types:
- 2f6c5d21:
    slots:    [('0', 1), ('1', 1), ('10', 1), ('11', 1), ('12', 1), ('13', 1), ('14', 1), ('15', 1), ('16', 1), ('17', 1), ('18', 1), ('19', 1), ('2', 1), ('20', 1), ('21', 1), ('22', 1), ('23', 1), ('24', 1), ('25', 1), ('26', 1), ('27', 1), ('28', 1), ('29', 1), ('3', 1), ('30', 1), ('31', 1), ('32', 1), ('33', 1), ('34', 1), ('35', 1), ('36', 1), ('37', 1), ('38', 1), ('39', 1), ('4', 1), ('40', 1), ('42', 1), ('43', 1), ('44', 1), ('45', 1), ('46', 1), ('47', 1), ('48', 1), ('49', 1), ('5', 1), ('50', 1), ('51', 1), ('52', 1), ('53', 1), ('54', 1), ('55', 1), ('56', 1), ('57', 1), ('58', 1), ('59', 1), ('6', 1), ('7', 1), ('8', 1), ('9', 1)]
    messages: ['10.03 - NODE - INFO - Node 7 appended SCPExternalize', '10.09 - Node 39 adopting externalized value for slot 1:', '10.11 - Node 26 adopting externalized value for slot 1:', '10.13 - Node 25 adopting externalized value for slot 1:', '10

In [331]:
def analyze_node_repeated_txs_with_msgs(df):
    """
    For each node, find transactions that are finalized
    in more than one slot.  For each such tx, record the
    slot *and* the first‑3‑words of the log message that
    reported it in that slot.
    
    Returns a dict:
      node -> {
        tx_hash -> [ (slot, msg_type), ... ]
      }
    """
    # intermediate store: node -> tx -> list of (slot,msg_type)
    temp = {}
    
    for _, row in df.iterrows():
        node = row['node name']
        slot = row['Slot']
        txs  = row['Finalised transactions']
        msg  = row['Externalize message']
        # first three words of the message
        msg_type = " ".join(msg.split()[:50])
        
        # initialize per‐node
        if node not in temp:
            temp[node] = {}
        
        for tx in txs:
            temp[node].setdefault(tx, []).append((slot, msg_type))
    
    # now filter to only those txes that appear in >1 distinct slot for each node
    result = {}
    for node, tx_dict in temp.items():
        repeated = {}
        for tx, slot_msgs in tx_dict.items():
            # collect distinct slots
            slots = { sm[0] for sm in slot_msgs }
            if len(slots) > 1:
                # keep all occurrences (slot,msg_type)
                repeated[tx] = slot_msgs
        if repeated:
            result[node] = repeated
    
    return result

# --- example usage ---
multi = analyze_node_repeated_txs_with_msgs(df)

for node, tx_info in multi.items():
    print(f"\nNode {node}:")
    for tx, occurrences in tx_info.items():
        print(f"  Transaction {tx!r} is in multiple slots:")
        for slot, msg_type in sorted(occurrences):
            print(f"    Slot {slot:>2} via “{msg_type}”")



Node 7:
  Transaction 'df92f547' is in multiple slots:
    Slot  7 via “129.68 - Node 7 adopting externalized value for slot 7: [Value, hash = 4314617724629887791, state = State.init, transactions = {[Transaction 34d4b4db time = 86.2962], [Transaction 2c973bbf time = 86.6354], [Transaction 3c681aac time = 76.8974], [Transaction fb0f4c8f time = 84.8483], [Transaction fb92d3f1 time = 90.1101], [Transaction 6c704ba6 time = 89.2025], [Transaction”
    Slot  8 via “180.07 - Node 7 adopting externalized value for slot 8: [Value, hash = -4641650456008855056, state = State.init, transactions = {[Transaction 98dc01e time = 142.8438], [Transaction e030d04b time = 103.3935], [Transaction 52b43122 time = 130.4801], [Transaction a12dc27 time = 126.8980], [Transaction 178a91cf time = 136.9712], [Transaction c1a192b6 time = 94.1966], [Transaction”
  Transaction 'fb92d3f1' is in multiple slots:
    Slot  7 via “129.68 - Node 7 adopting externalized value for slot 7: [Value, hash = 4314617724629887791

ADD INTERLEDGER CHECKS

In [332]:
def calculate_inter_ledger_agreement_time(df):
    df = df.sort_values(by='Timestamp of finalisation')
    time_diffs = df['Timestamp of finalisation'].diff().dropna()
    
    return time_diffs.mean()

avg_time = calculate_inter_ledger_agreement_time(df)
print(f"Average Inter-Ledger Agreement Time: {avg_time}")

Average Inter-Ledger Agreement Time: 0.3640709812108559


In [333]:
final_experiment_df = df[[
    "sequence number",
    "Timestamp of finalisation",
    "No. of finalised transactions",
    "no. of transactions not finalised"
]]

display(final_experiment_df)

KeyError: "['sequence number'] not in index"

In [ ]:
avg_difference = (final_experiment_df["no. of transactions not finalised"] - final_experiment_df["No. of finalised transactions"]).mean()

print(f"Average difference: {avg_difference}")

In [ ]:
avg_finalised = final_experiment_df["No. of finalised transactions"].mean()
avg_total = (final_experiment_df["No. of finalised transactions"] + 
             final_experiment_df["no. of transactions not finalised"]).mean()

finalised_percentage = (avg_finalised / avg_total) * 100 if avg_total != 0 else 0

print(f"Percentage of finalised transactions vs total: {finalised_percentage:.2f}%")